In [1]:
import numpy as np,pandas as pd,seaborn as sns,warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv(r'data.csv')
df.head()

,Shipment ID,Origin,Destination,Shipment Date,Planned Delivery Date,Actual Delivery Date,Vehicle Type,Distance (km),Weather Conditions,Traffic Conditions,Delayed
0,SHIP000000,Jaipur,Mumbai,2023-04-26,2023-05-01,2023-05-02,Trailer,1603,Rain,Light,Yes
1,SHIP000001,Bangalore,Delhi,2023-02-09,2023-02-13,2023-02-17,Trailer,1237,Storm,Moderate,Yes
2,SHIP000002,Mumbai,Chennai,2023-09-19,2023-09-25,2023-09-25,Truck,1863,Clear,Light,No
3,SHIP000003,Hyderabad,Ahmedabad,2023-04-01,2023-04-05,2023-04-05,Container,1374,Clear,Light,No
4,SHIP000004,Chennai,Kolkata,2023-11-24,2023-11-26,2023-11-28,Container,676,Clear,Heavy,Yes


In [47]:
df.shape

(20000, 11)

In [48]:
df.isnull().sum()

Shipment ID                0
Origin                     0
Destination                0
Shipment Date              0
Planned Delivery Date      0
Actual Delivery Date       0
Vehicle Type             597
Distance (km)              0
Weather Conditions         0
Traffic Conditions         0
Delayed                    0
dtype: int64

In [49]:
df.head()

,Shipment ID,Origin,Destination,Shipment Date,Planned Delivery Date,Actual Delivery Date,Vehicle Type,Distance (km),Weather Conditions,Traffic Conditions,Delayed
0,SHIP000000,Jaipur,Mumbai,2023-04-26,2023-05-01,2023-05-02,Trailer,1603,Rain,Light,Yes
1,SHIP000001,Bangalore,Delhi,2023-02-09,2023-02-13,2023-02-17,Trailer,1237,Storm,Moderate,Yes
2,SHIP000002,Mumbai,Chennai,2023-09-19,2023-09-25,2023-09-25,Truck,1863,Clear,Light,No
3,SHIP000003,Hyderabad,Ahmedabad,2023-04-01,2023-04-05,2023-04-05,Container,1374,Clear,Light,No
4,SHIP000004,Chennai,Kolkata,2023-11-24,2023-11-26,2023-11-28,Container,676,Clear,Heavy,Yes


In [50]:
drop_columns=["Shipment ID","Origin","Destination"]

In [51]:
df.drop(drop_columns,axis=1,inplace=True)

In [52]:
df["Actual Delivery Date"] = pd.to_datetime(df["Actual Delivery Date"])
df["Planned Delivery Date"] = pd.to_datetime(df["Planned Delivery Date"])
df["Shipment Date"] = pd.to_datetime(df["Shipment Date"])

In [53]:
 #df["Delivery Delay (days)"] = (df["Actual Delivery Date"] - df["Planned Delivery Date"]).dt.days
df["Planned Shipment Gap (days)"] = (df["Planned Delivery Date"] - df["Shipment Date"]).dt.days


In [54]:
drop_columns=["Actual Delivery Date","Planned Delivery Date","Shipment Date"]
df.drop(drop_columns,axis=1,inplace=True)

In [55]:
df.head(2)

,Vehicle Type,Distance (km),Weather Conditions,Traffic Conditions,Delayed,Planned Shipment Gap (days)
0,Trailer,1603,Rain,Light,Yes,5
1,Trailer,1237,Storm,Moderate,Yes,4


In [56]:
df.fillna({'Vehicle Type': df['Vehicle Type'].mode()[0]}, inplace=True)


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Vehicle Type                 20000 non-null  object
 1   Distance (km)                20000 non-null  int64 
 2   Weather Conditions           20000 non-null  object
 3   Traffic Conditions           20000 non-null  object
 4   Delayed                      20000 non-null  object
 5   Planned Shipment Gap (days)  20000 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 937.6+ KB


In [58]:
X =df.drop(columns=['Delayed'])

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,LabelEncoder
from datetime import datetime

In [60]:
clmn = ColumnTransformer(transformers=[
    # 1. Impute missing values
    ('encode_weather', OneHotEncoder(drop='first'), ['Weather Conditions', 'Traffic Conditions', 'Vehicle Type']),
    ('scale_distance', StandardScaler(), ['Distance (km)']),
],
remainder='passthrough')


X_transformed = clmn.fit_transform(X)

In [61]:
X.head(2)

,Vehicle Type,Distance (km),Weather Conditions,Traffic Conditions,Planned Shipment Gap (days)
0,Trailer,1603,Rain,Light,5
1,Trailer,1237,Storm,Moderate,4


In [62]:
X_transformed[0]

array([0.       , 1.       , 0.       , 1.       , 0.       , 0.       ,
       1.       , 0.       , 0.9628031, 5.       ])

In [63]:
clmn

ColumnTransformer(remainder='passthrough',
                  transformers=[('encode_weather', OneHotEncoder(drop='first'),
                                 ['Weather Conditions', 'Traffic Conditions',
                                  'Vehicle Type']),
                                ('scale_distance', StandardScaler(),
                                 ['Distance (km)'])])

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
X = X_transformed
y = df['Delayed']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=24)


In [67]:
X_train[0]


array([ 0.       ,  1.       ,  0.       ,  1.       ,  0.       ,
        0.       ,  0.       ,  1.       , -0.8289989,  2.       ])

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
model_log = LogisticRegression()
model_log.fit(X_train,y_train)
predictions_log = model_log.predict(X_test)



from sklearn.metrics import accuracy_score,classification_report

accuracy = accuracy_score(y_test, predictions_log)
print("Accuracy:", accuracy)


cross_val_log = cross_val_score(model_log, X_train, y_train, cv=5)
print(cross_val_log)



print(classification_report(y_test, predictions_log))



Accuracy: 0.91175
[0.914375  0.9215625 0.9159375 0.9225    0.9175   ]
              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1027
           1       1.00      0.88      0.94      2973

    accuracy                           0.91      4000
   macro avg       0.87      0.94      0.90      4000
weighted avg       0.93      0.91      0.92      4000



In [69]:
from sklearn.svm import SVC
model_svc = SVC()
model_svc.fit(X_train, y_train)
predictions_svc = model_svc.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions_svc)
print("Accuracy:", accuracy)


cross_val_svc = cross_val_score(model_svc, X_train, y_train, cv=5)
print(cross_val_svc)


print(classification_report(y_test, predictions_svc))


Accuracy: 0.91175
[0.914375  0.921875  0.915625  0.9221875 0.9175   ]
              precision    recall  f1-score   support

           0       0.74      1.00      0.85      1027
           1       1.00      0.88      0.94      2973

    accuracy                           0.91      4000
   macro avg       0.87      0.94      0.90      4000
weighted avg       0.93      0.91      0.92      4000



In [70]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
predictions_rf = model_rf.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions_rf)
print("Accuracy:", accuracy)


cross_val_rf = cross_val_score(model_rf, X_train, y_train, cv=5)
print(cross_val_rf)


print(classification_report(y_test, predictions_rf))

Accuracy: 0.874
[0.875625  0.886875  0.876875  0.8828125 0.87625  ]
              precision    recall  f1-score   support

           0       0.74      0.79      0.76      1027
           1       0.92      0.90      0.91      2973

    accuracy                           0.87      4000
   macro avg       0.83      0.85      0.84      4000
weighted avg       0.88      0.87      0.88      4000



In [71]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

model_xgb = XGBClassifier()
model_xgb.fit(X_train, y_train)
predictions_xgb = model_xgb.predict(X_test)

accuracy = accuracy_score(y_test, predictions_xgb)
print("Accuracy:", accuracy)

cross_val_xgb = cross_val_score(model_xgb, X_train, y_train, cv=5)
print(cross_val_xgb)

print(classification_report(y_test, predictions_xgb))


Accuracy: 0.90625
[0.9084375 0.9178125 0.9159375 0.9196875 0.9125   ]
              precision    recall  f1-score   support

           0       0.74      0.97      0.84      1027
           1       0.99      0.88      0.93      2973

    accuracy                           0.91      4000
   macro avg       0.87      0.93      0.89      4000
weighted avg       0.93      0.91      0.91      4000



In [72]:
import pickle

# Save the transformations
with open('column_transformer.pkl', 'wb') as f:
    pickle.dump(clmn, f)

# Save the Logistic Regression model
with open('model_log.pkl', 'wb') as f:
    pickle.dump(model_log, f)

In [73]:
df_2 = pd.read_csv(r'AI ML Internship Training Data.xlsx - freight_delivery_realistic_data.csv')
df_2.drop(columns=["Shipment ID"],inplace = True)
df_2.head()

,Origin,Destination,Shipment Date,Planned Delivery Date,Actual Delivery Date,Vehicle Type,Distance (km),Weather Conditions,Traffic Conditions,Delayed
0,Jaipur,Mumbai,2023-04-26,2023-05-01,2023-05-02,Trailer,1603,Rain,Light,Yes
1,Bangalore,Delhi,2023-02-09,2023-02-13,2023-02-17,Trailer,1237,Storm,Moderate,Yes
2,Mumbai,Chennai,2023-09-19,2023-09-25,2023-09-25,Truck,1863,Clear,Light,No
3,Hyderabad,Ahmedabad,2023-04-01,2023-04-05,2023-04-05,Container,1374,Clear,Light,No
4,Chennai,Kolkata,2023-11-24,2023-11-26,2023-11-28,Container,676,Clear,Heavy,Yes


In [74]:
columns =["Origin","Destination","Vehicle Type","Weather Conditions","Traffic Conditions"]

for i in columns:
    print(df_2[i].unique())

['Jaipur' 'Bangalore' 'Mumbai' 'Hyderabad' 'Chennai' 'Kolkata' 'Lucknow'
 'Delhi' 'Ahmedabad' 'Pune']
['Mumbai' 'Delhi' 'Chennai' 'Ahmedabad' 'Kolkata' 'Lucknow' 'Bangalore'
 'Pune' 'Jaipur' 'Hyderabad']
['Trailer' 'Truck' 'Container' 'Lorry' nan]
['Rain' 'Storm' 'Clear' 'Fog']
['Light' 'Moderate' 'Heavy']
